# Sentiment Analysis
In this notebook, we perform very primitive sentiment analysis on the lyrics for each song in our dataset.

In [97]:
import re
from collections import Counter
import os
import pandas as pd

In [54]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize
from nltk.corpus import stopwords

In [65]:
def clean_line(x):
    x = ' '.join(x.split())
    x = "".join(re.sub(r'[^\w\s\']','',x)).lower()
    return(x)
    

In [ ]:
# https://towardsdatascience.com/how-to-analyze-emotions-and-words-of-the-lyrics-from-your-favorite-music-artist-bbca10411283

In [79]:
input_dir = '/Volumes/SECONDDRIVE/prog/ug/lyric_lists/'
decade_dirs = [f.path for f in os.scandir(input_dir) if f.is_dir() and f.path.split("/")[-1].isnumeric()]

In [101]:
sid = SentimentIntensityAnalyzer()
song_info = []
for decade_dir in decade_dirs:
    
    decade = decade_dir.split("/")[-1]

    print(decade)
    filelist = [input_dir+ decade + "/" + x for x in os.listdir(input_dir+ decade + "/")]

    for fname in filelist:    
        with open (fname, "r") as myfile:
            data = [clean_line(x) for x in myfile.readlines()]
            data = [x for x in data if len(x) >=3]

                   
        word_occurrence = Counter(" ".join(data).split(" "))
        better_word_occurrence = Counter([x for x in " ".join(data).split(" ") if x not in set(stopwords.words('english'))])
        info_dict = {
            'song': fname.split(".")[0].split("/")[-1].split("_")[1].replace("-", " "),
            'artist': fname.split(".")[0].split("/")[-1].split("_")[0].replace("-", " "),
            'decade': decade,
            'total_lines': len(data),
            'unique_lines': len(list(set(data))),
            'total_words': len(" ".join(data).split()),
            'unique_words': len(" ".join(list(set(data))).split()),
            'top_common_words': ",".join([x[0] for x in word_occurrence.most_common(10)]),
            'top_uncommon_words': ",".join([x[0] for x in better_word_occurrence.most_common(10)])
        }
        
        info_dict.update(sid.polarity_scores(" ".join(data)))
        song_info.append(info_dict)
                   

1970
1980
1990
2000
2010


In [102]:
pd.DataFrame.from_dict(song_info).to_csv("../Output/lyrics.csv", index=False)